In [52]:
%%bash
service mongod status


mongod start/running, process 968


In [53]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
client.database_names()

['admin', 'data_diggers', 'enron', 'local']

## Reading UK Universities ranking data

In [37]:
#import the excel file
import xlrd
from collections import defaultdict
uniRank_workbook=xlrd.open_workbook('../normalize_dataset/UK_Universities_ranking.xls', on_demand = True)
uniRank_sheet=uniRank_workbook.sheet_by_index(0)

# Create database and collection objects for UK Universities ranking
db = client.data_diggers
UniRankCollection = db.UniRank
UniRankCollection.drop()
ur=list()
for i in range(1,uniRank_sheet.nrows-3):
    row1 = uniRank_sheet.row_values(i)
    dic1 = {'city_name':row1[0],'Number_of_universities':row1[1],'best_ranking':row1[2],'average_ranking':row1[3],'total_jobs_normalized_value':row1[4]}
    ur.append(dic1)
# print (ur)
UniRankCollection.insert_many(ur)
UniRankCollection.count()

56

## Reading Total Jobs data

In [38]:
#import the excel file
import xlrd
from collections import defaultdict
totalJobs_workbook=xlrd.open_workbook('../normalize_dataset/total_jobs_new Z Score.xls', on_demand = True)
totalJobs_sheet=totalJobs_workbook.sheet_by_index(0)

# Create database and collection objects for Total jobs
db = client.data_diggers
totalJobsCollection = db.TotalJobs
totalJobsCollection.drop()
l=list()
for i in range(1,totalJobs_sheet.nrows):
    row = totalJobs_sheet.row_values(i)
    dic2 = {'city_name':row[0],'total_jobs':row[2],'total_jobs_normalized_value':row[3]}
    l.append(dic2)
#print (l)
totalJobsCollection.insert_many(l)
totalJobsCollection.count()

62

## Reading Road Traffic data

In [39]:
#import the excel file
import xlrd
from collections import defaultdict
roadTraffic_workbook=xlrd.open_workbook('../normalize_dataset/road traffic.xls', on_demand = True)
roadTraffic_sheet=roadTraffic_workbook.sheet_by_index(0)
# Create database and collection objects for road traffic and delay caused by it
db = client.data_diggers
roadTrafficCollection = db.roadTraffic
db.drop_collection('roadTraffic')
rt=list()
for i in range(1,roadTraffic_sheet.nrows-3):
    row2 = roadTraffic_sheet.row_values(i)
    if row2[0]!='':
        dic3 = {'city_name':row2[0].split(' UA')[0].split(' (Met County)')[0],'Delay 2015':row2[2],'Delay 2016':row2[3],'Delay_2015_normalized_value':row2[5],'Delay_2016_normalized_value':row2[6]}
        rt.append(dic3)
#print (l)
roadTrafficCollection.insert_many(rt)
roadTrafficCollection.count()

159

## Number of people exposed to road traffic noise 55dB

In [40]:
#import the excel file
import xlrd
from collections import defaultdict
trafficNoise_workbook=xlrd.open_workbook('../normalize_dataset/road_traffic_noise.xls', on_demand = True)
trafficNoise_sheet=trafficNoise_workbook.sheet_by_index(0)
# Create database and collection objects for Number of people exposed to road traffic noise 55 dB
db = client.data_diggers
trafficNoiseCollection = db.trafficNoise
db.drop_collection('trafficNoise')
tn=list()
for i in range(1,trafficNoise_sheet.nrows):
    row3 = trafficNoise_sheet.row_values(i)
    dic4 = {'city_name':row3[0],'Number of people exposed to traffic noise':row3[1],'Normalized value of Number of people exposed to traffic noise':row3[2],}
    tn.append(dic4)
#print (l)
trafficNoiseCollection.insert_many(tn)
trafficNoiseCollection.count()

60

## Happiness ranking data

In [56]:
#import the excel file
import xlrd
from collections import defaultdict
happiness_workbook=xlrd.open_workbook('../normalize_dataset/1.happiness_geo.xls', on_demand = True)
happiness_sheet=happiness_workbook.sheet_by_index(1)
# Create database and collection objects for Happiness rankings by Country
db = client.data_diggers
happinessCollection = db.happinessRank
db.drop_collection('happinessRank')
hl=list()
for i in range(6,happiness_sheet.nrows):
    row4 = happiness_sheet.row_values(i)
    if row4[6]!= '':
        if row4[1][0] == ' ':
            row4[1] = row4[1][1:]
        dic5 = {'city_name':row4[1],'Average happiness rating':row4[2],'Sample size':row4[6],'Normalized value of happiness ranking':row4[8]}
        hl.append(dic5)
happinessCollection.insert_many(hl)
p = happinessCollection.find()
# for i in p:
#     print(i)

## population 

In [42]:
import csv
l=list()
filename ='../normalize_dataset/9.population.csv'
with open(filename) as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        rows=row[0].split('\t')
        l.append(rows)
        
db = client.data_diggers
collection =db.population_collection
population=list()
collection.drop()
for i in range(63):
    population.append({"city_name":l[i+1][0],"countries":l[i+1][1],"population":l[i+1][2],"N_population":l[i+1][3]})
collection.insert_many(population)
collection.count()

63

## The House price

In [43]:
db.drop_collection('houseprice_collection')
collection =db.houseprice_collection
collection.drop()
import csv
l=list()
s =set()
filename ='../normalize_dataset/7.UK-House price_Nor.csv'
with open(filename) as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        if row[0]!='Date':
            l.append({
                "city_name":row[1],
                "average_price":row[3],
                "Normalization":row[5]
            })
            s.add(row[1])
    for city_name in s:
        count = 0
        summ = 0
        nor_sum =0
        for city in l:
            if city['city_name'] == city_name:
                summ += float(city['average_price'])
                nor_sum += float(city['Normalization'])
                count += 1
        average_price = summ/float(count)
        normalization = nor_sum/float(count)
        collection.insert_one({"city_name":city_name,"average_price":average_price,"Normalization":normalization})
collection.count()

441

## gva_per_worker

In [44]:
collection = db.gva_collection
collection.drop()

import csv
l=list()
filename ='../normalize_dataset/10.gva_per_worker.csv'
with open(filename) as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        l.append({
            "city_name":row[0],
            "GVA_per_worker_per_month":row[1],
            "GVA_norm":row[2],
            "GVA_max_min_norm":row[3]    
        })
        collection.delete_one(l[0])
    for i in range(1,63):
        collection.insert_one(l[i])
collection.count()

62

## Unemployment

In [45]:
collection = db.unemployment
collection.drop()
import csv
l=list()
filename ='../normalize_dataset/11.unemployment_rate.csv'
with open(filename) as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        l.append({
            "city_name":row[0],
            "Unemployment_rate":row[1],
            "Unemployment_norm":row[2],
            "Unemployment_max_min_norm":row[3]    
        })
    collection.delete_one(l[0])
    for i in range(1,63):
        collection.insert_one(l[i])
collection.find_one()

{'Unemployment_max_min_norm': '0.3906976744',
 'Unemployment_norm': '-0.4370771187',
 'Unemployment_rate': '7.51',
 '_id': ObjectId('5a4d4cf77c4c7b78c0dc199e'),
 'city_name': 'Aberdeen'}

## reference

In [46]:
# Create database and collection objects for convenience
db = client.data_diggers
collection = db.reference
import json
with open('../cities.json', 'r') as f:
    cities = json.load(f)
collection.drop()
collection.insert_many(cities)
collection.count()

1742

## education

In [47]:
collection = db.education
import json
with open('../normalize_dataset/education.json', 'r') as f:
    education = json.load(f)

l = []
for edu in education:
    l.append({'city_name':edu,'school_number_normalize_maxmin':education[edu]})

collection.drop()
collection.insert_many(l)
collection.count()

1323

## hospital

In [48]:
collection = db.hospitals
import json
with open('../normalize_dataset/hospitals.json', 'r') as f:
    hospitals = json.load(f)

l = []
for hos in hospitals:
    l.append({'city_name':hos,'hospitals_number_normalize_maxmin':hospitals[hos]})

collection.drop()
collection.insert_many(l)
collection.count()

520

## pubs

In [49]:
collection = db.pubs
import json
with open('../normalize_dataset/pubs.json', 'r') as f:
    pubs = json.load(f)

l = []
for pub in pubs:
    l.append({'city_name':pub,'pubs_number_normalize_maxmin':pubs[pub]})
# l
collection.drop()
collection.insert_many(l)
collection.find_one()

{'_id': ObjectId('5a4d4cf87c4c7b78c0dc27dd'),
 'city_name': 'Mendip',
 'pubs_number_normalize_maxmin': 171}

## rail_way station

In [50]:
collection = db.railway_station
import json
with open('../normalize_dataset/railway_station.json', 'r') as f:
    stations = json.load(f)

l = []
for station in stations:
    l.append({'city_name':station,'stations_number_normalize_maxmin':stations[station]})

collection.drop()
collection.insert_many(l)
collection.count()

389

## weather

In [51]:
#import the excel file
import xlrd
from collections import defaultdict


workbook_norm =xlrd.open_workbook('../normalize_dataset/WeatherNorm.xls', on_demand = True)
sheet_norm=workbook_norm.sheet_by_index(0)    
workbook =xlrd.open_workbook('../clean_dataset/15.Weather.xls', on_demand = True)
sheet=workbook.sheet_by_index(0)    

weather_by_station={}
for i in range(1,sheet.nrows):
    weather_station_inf = sheet.row_values(i)
    weather_station_inf_norm = sheet_norm.row_values(i)
    weather_by_station[weather_station_inf[0]] = {"SummerDay_ave_temperature":weather_station_inf[3],
                                                  "SummerDay_ave_temperature_norm":weather_station_inf_norm[3],
                                                  "SummerNight_ave_temperature":weather_station_inf[4],
                                                  "SummerNight_ave_temperature_norm":weather_station_inf_norm[4],
                                                  "WinterDay_ave_temperature":weather_station_inf[5],
                                                  "WinterDay_ave_temperature_norm":weather_station_inf_norm[5],
                                                  "WinterNight_ave_temperature":weather_station_inf[6],
                                                  "WinterNight_ave_temperature_norm":weather_station_inf_norm[6],
                                                  
                                                  "FrostDay_perYear":weather_station_inf[7],
                                                  "FrostDay_perYear_norm":weather_station_inf_norm[7],
                                                  "Rainfall_perMon":weather_station_inf[8],
                                                  "Rainfall_perMon_norm":weather_station_inf_norm[8],
                                                  "Sunshine_perMon":weather_station_inf[9],
                                                  "Sunshine_perMon_norm":weather_station_inf_norm[9]
                                                 }
# print(weather_by_station)
import json
with open('../normalize_dataset/weatherStationsToCities.json', 'r') as f:
    stations = json.load(f)
    
l = []
for station in stations:
    l.append({"city_name":station,
              "SummerDay_ave_temperature":weather_by_station[stations[station]]["SummerDay_ave_temperature"],
              "SummerDay_ave_temperature_norm":weather_by_station[stations[station]]["SummerDay_ave_temperature_norm"],
              "SummerNight_ave_temperature":weather_by_station[stations[station]]["SummerNight_ave_temperature"],
              "SummerNight_ave_temperature_norm":weather_by_station[stations[station]]["SummerNight_ave_temperature_norm"],
              "WinterDay_ave_temperature":weather_by_station[stations[station]]["WinterDay_ave_temperature"],
              "WinterDay_ave_temperature_norm":weather_by_station[stations[station]]["WinterDay_ave_temperature_norm"],
              "WinterNight_ave_temperature":weather_by_station[stations[station]]["WinterNight_ave_temperature"],
              "WinterNight_ave_temperature_norm":weather_by_station[stations[station]]["WinterNight_ave_temperature_norm"],
              "FrostDay_perYear":weather_by_station[stations[station]]["FrostDay_perYear"],
              "FrostDay_perYear_norm":weather_by_station[stations[station]]["FrostDay_perYear_norm"],
              "Rainfall_perMon":weather_by_station[stations[station]]["Rainfall_perMon"],
              "Rainfall_perMon_norm":weather_by_station[stations[station]]["Rainfall_perMon_norm"],
              "Sunshine_perMon":weather_by_station[stations[station]]["Sunshine_perMon"],
              "Sunshine_perMon_norm":weather_by_station[stations[station]]["Sunshine_perMon_norm"]
             })

db = client.data_diggers
collection = db.weather
# print(l)
collection.drop()
collection.insert_many(l)
collection.count()

1742

# check who database

In [32]:
db = client.data_diggers
db.collection_names()

['weather',
 'houseprice_collection',
 'education',
 'training_data',
 'unemployment',
 'trafficNoise',
 'hospitals',
 'population_collection',
 'gva_collection',
 'UniRank',
 'happinessRank',
 'railway_station',
 'pubs',
 'roadTraffic',
 'reference',
 'TotalJobs']